#### About the Dataset:

##### id: unique id for a news article
##### title: the title of a news article
##### author: author of the news article
##### text: the text of the article; could be incomplete
##### label: a label that marks whether the news article is real or fake:
    1: Fake news
    0: real News

In [228]:
# importing the dependencies

import pandas as pd
import numpy as np 
import re #regular expression
from nltk.corpus import stopwords #nltk= natural language toolkit and corpus is where the data is stored , stopwards are not so useful words like articles
from nltk.stem.porter import PorterStemmer #for stemming 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score




In [229]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Priyanka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [230]:
#printing stopwords in english 
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

#### Data Pre-processing 

In [231]:
#loading the dataset into pandas dataframe

news_dataset= pd.read_csv('../ Fake News Prediction using Machine Learning with Python and logistic regression model/Data/train.csv')

In [232]:
news_dataset.shape

(20800, 5)

In [233]:
#printing 1st 5 rows of dataset
news_dataset.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [234]:
#checking if dataset has missing values
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [235]:
#replacing null values with empty string
news_dataset= news_dataset.fillna('')

In [236]:
#combining/merging title and author name together for better accuracy 
news_dataset['content']=news_dataset['author']+' ' +news_dataset['title']

In [237]:
news_dataset.head(5)

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [256]:
if 'content' not in news_dataset.columns or 'label' not in news_dataset.columns:
    raise ValueError("The dataset must contain 'content' and 'label' columns.")

In [257]:
#seperating data and label 
X=news_dataset.drop(['label'],axis=1)
Y=news_dataset['label']

In [258]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [259]:
#stemming 
port_stem = PorterStemmer()

In [260]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ',content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content



In [261]:

news_dataset['content'] = news_dataset['content'].apply(stemming)

In [264]:
print(news_dataset['content'])


0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [265]:
print("Sample of processed content:\n", news_dataset['content'].head())


Sample of processed content:
 0    darrel lucu hou dem aid even see comey letter ...
1    daniel j flynn flynn hillari clinton big woman...
2               consortiumnew com truth might get fire
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoy iranian woman jail fiction unpu...
Name: content, dtype: object


In [274]:
# Prepare the data
X = news_dataset['content'].values  # Text data
Y = news_dataset['label'].values  # Labels
print("Original Y shape:", Y.shape)

Original Y shape: (20800,)


In [275]:
# Ensure Y is a dense 1D array (if necessary)
Y = Y.ravel()  # Flatten if Y is a 2D array


In [276]:
print(X)
print(Y)

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [277]:
Y.shape

(20800,)

In [278]:
#computers cannot understand text so we convert text to meaningful number ,here we vectorize
#converting textual data to numerical data

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(news_dataset['content'])
print("Shape of X after vectorization:", X.shape)  # Should reflect (n_samples, n_features)

Shape of X after vectorization: (20800, 16984)


In [292]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210584 stored elements and shape (20800, 16984)>
  Coords	Values
  (0, 3567)	0.3598939188262559
  (0, 8822)	0.3635963806326075
  (0, 6930)	0.21874169089359144
  (0, 3759)	0.2705332480845492
  (0, 263)	0.27010124977708766
  (0, 4917)	0.233316966909351
  (0, 13348)	0.2565896679337957
  (0, 2933)	0.2468450128533713
  (0, 8546)	0.29212514087043684
  (0, 7612)	0.24785219520671603
  (0, 2464)	0.3676519686797209
  (0, 15553)	0.28485063562728646
  (1, 3535)	0.2653147533915268
  (1, 5440)	0.7186013955384664
  (1, 6744)	0.19152496072048605
  (1, 2790)	0.19208753385709676
  (1, 1481)	0.2957471154505952
  (1, 16656)	0.3025156488372128
  (1, 2206)	0.36915639258038363
  (1, 1877)	0.15614790568229528
  (2, 3072)	0.46097489583229645
  (2, 2917)	0.3179886800654691
  (2, 15478)	0.41544962664721613
  (2, 9532)	0.49351492943649944
  (2, 5903)	0.3474613386728292
  :	:
  (20797, 9500)	0.17463635692029988
  (20797, 6969)	0.21809398920480086
  (207

In [293]:
#splitting the data into training and testing data

X_train,Y_train,X_test,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)


In [294]:
# Ensure Y is a dense 1D array
Y = Y.flatten() if len(Y.shape) > 1 else Y  # Flatten if Y is a 2D array

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

# Check the shapes before fitting
print("X_train shape:", X_train.shape)  # Should be (n_samples, n_features)
print("Y_train shape:", Y_train.shape)  # Should be (n_samples,)

# Ensure Y_train is a 1D array
Y_train = Y_train.ravel()  # Flatten if necessary

# Final shape check
print("Adjusted Y_train shape:", Y_train.shape)  # Should be (n_samples,)


X_train shape: (16640, 16984)
Y_train shape: (16640,)
Adjusted Y_train shape: (16640,)


In [295]:
#training logistic regression model(sigmoid function)

model = LogisticRegression(max_iter=1000)  # Increase max_iter if needed
model.fit(X_train, Y_train)
print("Model trained successfully")

Model trained successfully


In [296]:
#Evaluation
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [297]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9863581730769231


In [298]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [299]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9790865384615385


In [300]:
#making a predictive system
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [301]:
print(Y_test[3])

0
